In [1]:
import requests
import os
import pandas as pd
import time
tokenCanvas = os.environ['tokenCanvas']

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
}

def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')      # Kommentar: denne er med BERRE for å skape balanse i .org-fila: \"
    return link_header_dict


def sis_import(filnamn):
    url = "https://hvl.instructure.com/api/v1/accounts/1/sis_imports"
    filar = {'attachment': open(filnamn, 'rb')}
    r = requests.post(url, headers=headers, files=filar)
    resultat = r.json()
    vent = True
    forseinking = 5
    while vent:
    # Her venter eg til alt er ferdig før eg tar neste campus
        jobb_id = resultat['id']
        time.sleep(forseinking)
        url = f"https://hvl.instructure.com/api/v1/accounts/1/sis_imports/{jobb_id}"
        r = requests.get(url, headers=headers)
        resultat = r.json()
        if resultat['progress']==100:
            print("Ferdig.")
            vent = False
        else:
            print(f"Arbeider framleis, {resultat['progress']} % ferdig.")

# Avslutte manuelt innlagt i eldre emne

## Problemstilling
Når vi legg inn brukarar manuelt i emne vert dei liggande aktive etter at emneet er avslutta, for det vert ikkje sendt melding om å avslutte innmeldinga for desse, slik det vert gjort for dei som kjem frå FS.

## Løysing
Eg reknar alle emne som har `enrollment_term_id` i lista over `avslutt_semestre` som avslutta. Det er desse som får "avslutt"-melding frå FS våren 2025 (kanskje). 
Så går eg gjennom denne lista (ca. 16000 emne) og finn alle personar som framleis er aktive. 

## CD2
Eg henter først alle `courses` frå CD2, for å sjå kva eg får ut.

In [3]:
alle_emne = pd.read_csv("courses.csv")

Så må eg finne alle terminane vi har, slik at eg kan plukke ut alle emne som skal avsluttast (emne som 30.6.2025 har "2024 VÅR" som siste termin). Så kan eg jo gjere dette for andre semestre også, men det seinare.

In [4]:
url = "https://hvl.instructure.com/api/v1/accounts/1/terms"
dr_liste = []
hentmeir = True
while hentmeir:
    print(url)
    respons = requests.get(url, headers=headers, params=params)
    if 200 <= respons.status_code < 300:
        data = respons.json()
        hentmeir = "next" in respons.headers['link']
        if hentmeir:
            url = finn_rel(respons.headers['link'])['next']
        dataliste = []
# Den neste linja varierer frå endepunkt til endepunkt
        for element in data['enrollment_terms']:
            dataliste.append(element)
        df = pd.DataFrame(dataliste, columns=['id', 'name'])
        dr_liste.append(df)
alledata = pd.concat((df for df in dr_liste if not df.empty), ignore_index=True)

avslutt = alledata[alledata['name'].str.endswith('2023 HØST')]['id']

https://hvl.instructure.com/api/v1/accounts/1/terms
https://hvl.instructure.com/api/v1/accounts/1/terms?page=2&per_page=100


Så kan eg leite etter emne som høyrer til i kvar termin:

In [5]:
avslutt_emne = []
for termin in avslutt:
    avslutt_emne.append(alle_emne[alle_emne['value.enrollment_term_id'] == termin]['key.id'].to_list())
    print(f"Emne som skal avsluttast for termin {termin}: {len(alle_emne[alle_emne['value.enrollment_term_id'] == termin])}")

Emne som skal avsluttast for termin 83: 1
Emne som skal avsluttast for termin 198: 0
Emne som skal avsluttast for termin 205: 2
Emne som skal avsluttast for termin 212: 31
Emne som skal avsluttast for termin 215: 7
Emne som skal avsluttast for termin 220: 1137


In [9]:
deaktiver = []
for emnekode in avslutt_emne[5][100:]:
    print(emnekode)
    try:
        url = f"https://hvl.instructure.com/api/v1/courses/{emnekode}/enrollments"
        innmeldingar = requests.get(url, headers=headers, params=params).json()
        for i in innmeldingar:
            if i['enrollment_state'] == 'active':
                if i['sis_user_id'] != None:
                    if i['role'] == "StudentEnrollment":
                        deaktiver.append([i['sis_user_id'],i['sis_section_id'],"Student","completed"])
                    if i['role'] == "TeacherEnrollment":
                        deaktiver.append([i['sis_user_id'],i['sis_section_id'],"Teacher","completed"])
    except:
        print(f"Feil ved {emnekode}")
pd.DataFrame(deaktiver, columns=['user_id', 'section_id', 'role', 'status']).to_csv("deaktiver.csv", index=False)

26208
26206
26201
26200
26268
26263
26267
26264
26259
26257
26252
26255
26253
26285
26289
26288
26286
26280
26273
26272
26271
26278
26276
26227
26226
26229
26221
26220
26216
26213
26219
26218
26217
26210
26248
26246
26241
26243
26235
26234
26231
26187
26175
26172
26179
26178
26177
26176
26197
26196
26198
26193
26191
26190
26303
26301
26305
26300
26326
26325
26328
26315
26314
26312
26316
26311
26310
26384
26383
26387
26386
26380
26378
26371
26370
26376
26395
26394
26393
26399
26397
26396
26390
26348
26346
26340
26341
26337
26335
26334
26339
26338
26333
26330
26362
26361
26366
26363
26357
26350
26354
26352
26296
26294
26297
26291
25915
25914
25918
25917
25911
25903
25908
25907
25900
25974
25971
25975
25970
25969
25968
25962
25966
25990
25984
25989
25986
25980
25936
25935
25930
25934
25931
25929
25920
25959
25958
25950
25956
25955
25953
25947
25946
25578
25570
25573
25571
25567
25566
25565
25569
25599
25592
25595
25586
25581
25580
25585
25583
25534
25532
25538
25537
25536
25528
25521
2552

In [ ]:
sis_import("deaktiver.csv")